In [8]:
import json, random, rouge, itertools
from tqdm import tqdm
from shroom_classifier_usp import ShroomClassifier

In [2]:
demo_generating_dataset = json.load(open('train.model-agnostic.json', 'r'))

In [3]:
dm_unlabelled_datapoints = random.sample([ dp for dp in demo_generating_dataset if dp['task'] == "DM" ], 64)
pg_unlabelled_datapoints = random.sample([ dp for dp in demo_generating_dataset if dp['task'] == "PG" ], 64)
mt_unlabelled_datapoints = random.sample([ dp for dp in demo_generating_dataset if dp['task'] == "MT" ], 64)
cp = ShroomClassifier(temperature=0.7)

In [4]:
dm_unlabelled_datapoints

[{'hyp': '(uncountable) The quality of being sentient.',
  'tgt': 'The state or quality of being sentient; possession of consciousness or sensory awareness.',
  'src': "The science of animal <define> sentience </define> is far from a firm one ; there 's no way of knowing exactly what any animal is feeling .",
  'ref': 'tgt',
  'task': 'DM',
  'model': ''},
 {'hyp': 'Of or pertaining to bisexuality.',
  'tgt': 'Negative towards bisexuality; biphobic.',
  'src': 'The epistemic erasure of bisexuality manages to uphold <define> binegative </define> attitudes and beliefs , which in turn can unwittingly be internalised by bisexual \xa0 ...',
  'ref': 'tgt',
  'task': 'DM',
  'model': ''},
 {'hyp': 'Alternative form of pre-blues',
  'tgt': 'From the period before blues music.',
  'src': 'Even among the ( rapidly disappearing ) survivors of his generation , Yank Rachell is nearly alone in his persistent use of traditional , even <define> preblues </define> forms .',
  'ref': 'tgt',
  'task': '

In [5]:
for dp in tqdm(dm_unlabelled_datapoints):
    dp.update(cp.classify(dp["task"], dp["src"], dp["tgt"], dp["hyp"], dp["ref"]))

100%|██████████| 64/64 [27:32<00:00, 25.82s/it]


In [7]:
dm_unlabelled_datapoints[0]

{'hyp': '(uncountable) The quality of being sentient.',
 'tgt': 'The state or quality of being sentient; possession of consciousness or sensory awareness.',
 'src': "The science of animal <define> sentience </define> is far from a firm one ; there 's no way of knowing exactly what any animal is feeling .",
 'ref': 'tgt',
 'task': 'DM',
 'model': '',
 'predictions': ['Not Hallucination',
  'Not Hallucination',
  'Not Hallucination',
  'Not Hallucination',
  'Not Hallucination'],
 'predicted': 'Not Hallucination',
 'predicted_p': 0.0,
 'timestamp': '2023-11-06T01:30:07Z',
 'rationale_0': 'The output is not a hallucination. It correctly identifies sentience as the quality of being sentient, which aligns with the target definition that refers to the possession of consciousness or sensory awareness. However, it is a simplified version and leaves out some details found in the target.',
 'rationale_1': "The output is not a hallucination. It correctly captures the basic essence of the term 'se

In [19]:
def average_pairwise_rouge(texts):
    r = rouge.Rouge()
    pairwise_combinations = [[a, b] for a, b in itertools.combinations(rationales, 2)]
    hyps, refs = map(list, zip(*pairwise_combinations))
    return r.get_scores(hyps, refs, avg=True)['rouge-1']['f']
    

In [20]:
rationales = [ 
    dm_unlabelled_datapoints[0]["rationale_0"], 
    dm_unlabelled_datapoints[0]["rationale_1"], 
    dm_unlabelled_datapoints[0]["rationale_2"], 
    dm_unlabelled_datapoints[0]["rationale_3"], 
    dm_unlabelled_datapoints[0]["rationale_4"], 
]

average_pairwise_rouge(rationales)

0.5271451850885072

In [ ]:
pg_unlabelled_datapoints

In [ ]:
mt_unlabelled_datapoints